# Kodutöö 7: Interaktiivne näotuvastus koos filtrite ja juturobotiga

In [ ]:
# Vajadusel eemalda kommentaarid, et installida vajalikud moodulid
#!pip install numpy
#!pip install opencv-python
#!pip install estnltk

In [ ]:
import threading
import time
import cv2 as cv
import numpy as np
import chatbot as ai

face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")

## Kasutajasisendi töötlemine ja filtri rakendamine

Järgnevad funktsioonid töötavad koos, et luua interaktiivne kasutajakogemus, kus kasutaja saab valida ja rakendada virtuaalseid filtreid. `inputWorker` funktsioon töötleb kasutaja sisendi ja genereerib vastuseid, juhindudes kasutaja soovidest. `overlayFilter` ja `filterImage` funktsioonid vastutavad filtrite haldamise eest, määrates filtri asukoha ja suuruse.

In [ ]:
# Tegeleb kasutaja poolt saadud sisenditega
def inputWorker(ref):
    print(ai.get_intro())
    time.sleep(0.5)
    while True:
        user_input = input("Kasutaja: ")
        ref[0], response = ai.get_response(user_input, ref[0])
        print(response)
        time.sleep(0.5)

        if ref[0] == "exit":
            break

In [ ]:
# Paneb filtri pildile
def overlayFilter(image, x, y, w, h, filter_img):
    # Võtame overlay pildist alpha kanali välja
    alpha = filter_img[:, :, 3]
    alpha = cv.merge([alpha, alpha, alpha])

    cropped_image = image[y:y + h, x:x + w]

    # Võtame overlay pildist BGR kanalid välja
    bgr = filter_img[:, :, 0:3]

    # Kui alpha on 0, vali taust, kui mitte, vali meie overlay filter
    image[y:y + h, x:x + w] = np.where(alpha == 0, cropped_image, bgr)
    return image

In [ ]:
# Valib vastava filtri, määrab selle suuruse ning paneb selle pildile
def filterImage(image, x, y, w, filter_name):
    file = filter_name.replace("ä", "a").replace("ö", "o").replace("õ", "o").replace("ü", "u")

    filter_img = cv.imread(f"filters/{file}.png", cv.IMREAD_UNCHANGED)

    fw = w
    fh = int(filter_img.shape[0] * w / float(filter_img.shape[1]))

    if filter_name in ai.get_head_filters():
        if filter_name == "deemon":
            y = y - int(w * 0.4)
        else:
            y = y - int(w * 0.6)
    elif filter_name in ai.get_eye_filters():
        y = y + int(w * 0.3)
    elif filter_name in ai.get_face_filters():
        if filter_name == "mask":
            y = y + int(w * 0.5)
        elif filter_name == "kloun":
            y = y + int(w * 0.2)
        elif filter_name == "koer":
            y = y - int(w * 0.1)
        elif filter_name == "karu":
            y = y - int(w / 4)
        elif filter_name == "vunts":
            x = x + int(w / 4)
            y = y + int(w * 0.7)
            fw = int(w / 2)
            fh = int((filter_img.shape[0] * fw) / float(filter_img.shape[1]))

    filter_img = cv.resize(filter_img, (fw, fh))

    # Kui filter läheb ekraanist välja, siis lõikame vastavalt vähemaks
    if y < 0:
        filter_img = filter_img[abs(y):]
        fh -= abs(y)
        y = 0

    if 0 <= x and x + fw < image.shape[1] and y + fh < image.shape[0]:
        return overlayFilter(image, x, y, fw, fh, filter_img)
    return image

## Põhiprogramm

Programm avab videovoo, käivitab eraldi lõime kasutaja sisendi töötlemiseks ja alustab tsüklit, mis loeb sisse kaamerasvideokaadreid. Iga kaadri puhul teostatakse näotuvastus ja rakendatakse eelpool defineeritud funktsioone, et leida inimeste näod, ja lisada filtrid, mis vastavad kasutaja poolt valitud võtmesõnadele. Kui kasutaja otsustab lõpetada või vajutab ESC klahvi, siis programm lõpetab videovoogu lugemise ja suleb kõik avatud aknad.

In [ ]:
video = cv.VideoCapture(0)
refs = [""]

threading.Thread(target=inputWorker, args=[refs]).start()

while True:
    result, img = video.read()

    # Flipib pildi ümber, et toimuks arusaadav peegeldus
    img = cv.flip(img, 1)

    # Näotuvastus
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_img)
    for (x, y, w, h) in faces:
        # cv.rectangle(img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        if refs[0] not in ["", "exit"]:
            img = filterImage(img, x, y, w, refs[0])

    cv.imshow("Camera", img)

    k = cv.waitKey(30) & 0xFF
    if k == 27:  # ehk ESC nupp
        break

    if refs[0] == "exit":
        break

video.release()
cv.destroyAllWindows()